# **Learn Computer Vision**
---

**Learning Project For Computer Vision**

Making a emotion detection using a CNN and Yolo

**This Computer Vision project is using a Micro-Expression Dataset**

[Micro-Expression Dataset (Kaggle Dataset)](https://www.kaggle.com/datasets/kmirfan/micro-expressions/data)

## **Table of Content**
1. Setup

2. Dataset Prep

3. EDA and Pre-Prosessing

4. CNN Model Devlop

5. YOLO Integration

6. Model training and evaluation

7. Real-time Interface

### **1. Setup**

#### 1.1. Install Req Library 

In [2]:
# use if needed to install packages
"""
%pip install opencv-python tensorflow keras numpy pandas matplotlib scikit-learn
%pip install yolov5 torch torchvision
%pip install pillow seaborn
pip install "tensorflow<2.11"
"""

'\n%pip install opencv-python tensorflow keras numpy pandas matplotlib scikit-learn\n%pip install yolov5 torch torchvision\n%pip install pillow seaborn\npip install "tensorflow<2.11"\n'

#### 1.2. Import Libraries 

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# For YOLO face detection
import torch
import yolov5

# Check for GPU availability
from tensorflow.python. client import device_lib

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs Available: ", len(gpus))
    print("GPUs name : ", device_lib.list_local_devices()[-1].physical_device_desc)
else:
    print("No GPU Available")

GPUs Available:  1
GPUs name :  device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:2b:00.0, compute capability: 8.6


### **2. Dataset Prep**

#### 2.1. Set Dataset Path

In [4]:
# Extract the dataset
dataset_path = 'Micro_Expressions'

# Check dataset structure
print("\nDataset structure:")
print(os.listdir(dataset_path))



Dataset structure:
['test', 'train']


#### 2.2. Explore Dataset Structure

because there already 2 folder in dataset so it will be sparate to train_dir and test_dir

In [5]:
# Define emotion classes (7 emotions in the dataset)
EMOTION_CLASSES = ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

# Define paths for train and test folders
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

# Verify all emotion classes exist
print("Verifying emotion classes in train folder:")
train_emotions = os.listdir(train_dir)
print(f"Found emotions: {sorted(train_emotions)}")

print("\nVerifying emotion classes in test folder:")
test_emotions = os.listdir(test_dir)
print(f"Found emotions: {sorted(test_emotions)}")

# Count images per emotion in train set
train_emotion_counts = {}
for emotion in EMOTION_CLASSES:
    emotion_path = os.path.join(train_dir, emotion)
    if os.path.isdir(emotion_path):
        train_emotion_counts[emotion] = len(os.listdir(emotion_path))

# Count images per emotion in test set
test_emotion_counts = {}
for emotion in EMOTION_CLASSES:
    emotion_path = os.path.join(test_dir, emotion)
    if os.path.isdir(emotion_path):
        test_emotion_counts[emotion] = len(os.listdir(emotion_path))

# Print counts
print("Images per emotion in TRAIN set:")
total_train = 0
for emotion in EMOTION_CLASSES:
    count = train_emotion_counts.get(emotion, 0)
    print(f"  {emotion:12s}: {count:4d} images")
    total_train += count

print("Images per emotion in TEST set:")
total_test = 0
for emotion in EMOTION_CLASSES:
    count = test_emotion_counts.get(emotion, 0)
    print(f"  {emotion:12s}: {count:4d} images")
    total_test += count

print(f"Total train images: {total_train}")
print(f"Total test images:  {total_test}")
print(f"Total images:       {total_train + total_test}")

Verifying emotion classes in train folder:
Found emotions: ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

Verifying emotion classes in test folder:
Found emotions: ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
Images per emotion in TRAIN set:
  anger       : 1411 images
  disgust     :  662 images
  fear        :  479 images
  happiness   : 1950 images
  neutral     :  644 images
  sadness     : 1369 images
  surprise    : 1085 images
Images per emotion in TEST set:
  anger       :  350 images
  disgust     :  160 images
  fear        :  120 images
  happiness   :  480 images
  neutral     :  160 images
  sadness     :  330 images
  surprise    :  260 images
Total train images: 7600
Total test images:  1860
Total images:       9460


#### 2.3. Visualize Sample Images

In [6]:
# Display sample images from each emotion (from train set)
num_emotions = len(EMOTION_CLASSES)
num_cols = 4
num_rows = (num_emotions + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(14, 10))
axes = axes.ravel()

for idx, emotion in enumerate(EMOTION_CLASSES):
    emotion_path = os.path.join(train_dir, emotion)
    image_files = os.listdir(emotion_path)
    
    if len(image_files) > 0:
        sample_image = image_files [0]
        img_path = os.path.join(emotion_path, sample_image)
        
        img = cv2.imread(img_path)
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            axes[idx].imshow(img_rgb)
            axes[idx].set_title(f'{emotion.capitalize()}', fontsize=12, fontweight='bold')
            axes[idx].axis('off')

# Hide unused subplots
for idx in range(len(EMOTION_CLASSES), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print(f"Sample image shape: {img_rgb.shape}")
print(f"Sample image dtype: {img_rgb.dtype}")


Sample image shape: (80, 80, 3)
Sample image dtype: uint8


### **3. Data Preprocessing**

#### 3.1. Load and Preprocess Images

In [7]:
IMG_SIZE = 80  # Micro-expressions dataset uses 80x80 images
BATCH_SIZE = 32

def load_images_and_labels(data_dir, emotion_classes, img_size=IMG_SIZE):
    """Load all images and labels from a directory (train or test)"""
    images = []
    labels = []
    
    # Calculate total files for progress tracking
    total_files = 0
    for emotion in emotion_classes:
        emotion_path = os.path.join(data_dir, emotion)
        if os.path.isdir(emotion_path):
            total_files += len(os.listdir(emotion_path))
    
    loaded = 0
    
    for emotion in emotion_classes:
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.isdir(emotion_path):
            print(f"Warning: {emotion_path} not found")
            continue
            
        for img_name in os.listdir(emotion_path):
            try:
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path)
                
                if img is not None:
                    # Resize image to standard size
                    img_resized = cv2.resize(img, (img_size, img_size))
                    
                    # Normalize pixel values to 0-1 range
                    img_normalized = img_resized / 255.0
                    
                    images.append(img_normalized)
                    labels.append(emotion)
                    loaded += 1
                    
                    if loaded % 100 == 0:
                        print(f"Loaded {loaded}/{total_files} images...", end='\r')
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue
    
    print(f"\nLoading complete! {loaded} images loaded")
    return np.array(images), np.array(labels)

# Load train and test datasets
print("TRAIN images")
X_train_full, y_train_full = load_images_and_labels(train_dir, EMOTION_CLASSES)
print(f"Loaded {len(X_train_full)} train images")

print("\nTEST images")
X_test_full, y_test_full = load_images_and_labels(test_dir, EMOTION_CLASSES)
print(f"Loaded {len(X_test_full)} test images")

print(f"\nImage shape: {X_train_full.shape}")
print(f"Unique emotions in train set: {np.unique(y_train_full)}")


TRAIN images
Loaded 7600/7600 images...
Loading complete! 7600 images loaded
Loaded 7600 train images

TEST images
Loaded 1800/1860 images...
Loading complete! 1860 images loaded
Loaded 1860 test images

Image shape: (7600, 80, 80, 3)
Unique emotions in train set: ['anger' 'disgust' 'fear' 'happiness' 'neutral' 'sadness' 'surprise']


#### 3.2. Encode Labels

In [8]:
# Encode emotion labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_full)
y_test_encoded = label_encoder.transform(y_test_full)

# Create a dictionary for emotion mapping
emotion_map = {i: emotion for i, emotion in enumerate(label_encoder.classes_)}
reverse_emotion_map = {emotion: i for i, emotion in emotion_map.items()}

print("Emotion mapping:")
for code, emotion in sorted(emotion_map.items()):
    print(f"  {code}: {emotion}")


Emotion mapping:
  0: anger
  1: disgust
  2: fear
  3: happiness
  4: neutral
  5: sadness
  6: surprise


#### 3.3. Split Training Dataset into Training and Validation

In [9]:
# Further split the training set into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded
)

# Use the original test set as our final test set
X_test = X_test_full
y_test = y_test_encoded

print("Data split summary:")
print(f"Training set:   {len(X_train)} images (80% of train folder)")
print(f"Validation set: {len(X_val)} images (20% of train folder)")
print(f"Test set:       {len(X_test)} images (all test folder)")

# Convert labels to categorical format for neural network
y_train_cat = keras.utils.to_categorical(y_train, num_classes=len(emotion_map))
y_val_cat = keras.utils.to_categorical(y_val, num_classes=len(emotion_map))
y_test_cat = keras.utils.to_categorical(y_test, num_classes=len(emotion_map))


Data split summary:
Training set:   6080 images (80% of train folder)
Validation set: 1520 images (20% of train folder)
Test set:       1860 images (all test folder)


#### 3.4. Data Augmentation

In [10]:
# Create data augmentation pipeline for training set
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
    fill_mode='nearest'
)

# Fit augmentation on training data
train_datagen.fit(X_train)


### **4. CNN Model Development**

#### 4.1. Build Custom CNN Model

In [11]:
def create_cnn_model(input_shape, num_classes):
    """Create a custom CNN model for emotion detection"""
    model = models.Sequential([
        # Block 1 - 32 filters
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2 - 64 filters
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3 - 128 filters
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Flatten and Dense layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create model
input_shape = (IMG_SIZE, IMG_SIZE, 3)
num_classes = len(emotion_map)
model = create_cnn_model(input_shape, num_classes)

print("CNN Model Architecture:")
print(model.summary())


CNN Model Architecture:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 80, 80, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 80, 80, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 80, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 80, 80, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 32)       0         
 )                                                               
                                

#### 4.2. Alternative: Transfer Learning with MobileNetV2

In [12]:
# For faster training on limited resources
def create_transfer_learning_model(input_shape, num_classes):
    """Create model using transfer learning with MobileNetV2"""
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Uncomment to use transfer learning instead of custom CNN
# model, base_model = create_transfer_learning_model(input_shape, num_classes)
# print("Transfer Learning Model created with MobileNetV2")


#### 4.3. Compile Model

In [13]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


### **5. Model Training**

#### 5.1. Define Callbacks

In [14]:
# Early stopping callback - stops training when validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

# Learning rate reduction callback - reduces learning rate when progress plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stopping, reduce_lr]

#### 5.2. Train the Model

In [15]:
# Train the model
EPOCHS = 100

history = model.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=BATCH_SIZE),
    validation_data=(X_val, y_val_cat),
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("Training completed successfully!")

Epoch 1/100
190/190 [==============================] - 17s 58ms/step - loss: 2.8656 - accuracy: 0.1531 - val_loss: 1.9855 - val_accuracy: 0.0914 - lr: 1.0000e-04
Epoch 2/100
190/190 [==============================] - 10s 53ms/step - loss: 2.7266 - accuracy: 0.1484 - val_loss: 1.9945 - val_accuracy: 0.1776 - lr: 1.0000e-04
Epoch 3/100
190/190 [==============================] - 10s 55ms/step - loss: 2.5999 - accuracy: 0.1681 - val_loss: 1.9739 - val_accuracy: 0.2217 - lr: 1.0000e-04
Epoch 4/100
190/190 [==============================] - 11s 58ms/step - loss: 2.5091 - accuracy: 0.1755 - val_loss: 2.0124 - val_accuracy: 0.1816 - lr: 1.0000e-04
Epoch 5/100
190/190 [==============================] - 11s 58ms/step - loss: 2.4847 - accuracy: 0.1765 - val_loss: 2.3697 - val_accuracy: 0.1855 - lr: 1.0000e-04
Epoch 6/100
190/190 [==============================] - 12s 62ms/step - loss: 2.3966 - accuracy: 0.1906 - val_loss: 2.4485 - val_accuracy: 0.1914 - lr: 1.0000e-04
Epoch 7/100
190/190 [=======

#### 5.3. Plot Training History

In [16]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTotal epochs trained: {len(history.history['loss'])}")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")


Total epochs trained: 36
Final training accuracy: 0.2062
Final validation accuracy: 0.1862


### **6. Model Evaluation**

#### 6.1. Evaluate on Test Set

In [17]:
# Evaluate model on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)

print("TEST SET EVALUATION RESULTS")
print(f"Test Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


# Get predictions
y_pred = model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Classification report
print("\nDetailed Classification Report:")
print(classification_report(
    y_test, 
    y_pred_classes, 
    target_names=[emotion_map[i] for i in range(len(emotion_map))],
    digits=4
))


TEST SET EVALUATION RESULTS
Test Loss:     1.9844
Test Accuracy: 0.2409

Detailed Classification Report:
              precision    recall  f1-score   support

       anger     0.2065    0.3429    0.2578       350
     disgust     0.0000    0.0000    0.0000       160
        fear     0.0000    0.0000    0.0000       120
   happiness     0.4048    0.2792    0.3305       480
     neutral     0.0000    0.0000    0.0000       160
     sadness     0.0000    0.0000    0.0000       330
    surprise     0.2046    0.7462    0.3212       260

    accuracy                         0.2409      1860
   macro avg     0.1166    0.1955    0.1299      1860
weighted avg     0.1719    0.2409    0.1787      1860



#### 6.2. Confusion Matrix

In [18]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=[emotion_map[i] for i in range(len(emotion_map))],
    yticklabels=[emotion_map[i] for i in range(len(emotion_map))],
    cbar_kws={'label': 'Count'},
    annot_kws={'size': 11}
)
plt.title('Confusion Matrix - Emotion Detection Model', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


#### 6.3. Save the Trained Model

In [19]:
# Create output directory if it doesn't exist
output_dir = './models'
os.makedirs(output_dir, exist_ok=True)

# Save emotion detection model
model_path = os.path.join(output_dir, 'emotion_detection_model.h5')
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save label encoder mapping
import pickle
encoder_path = os.path.join(output_dir, 'emotion_label_encoder.pkl')
with open(encoder_path, 'wb') as f:
    pickle.dump(emotion_map, f)
print(f"Label mapping saved to: {encoder_path}")

# Save model architecture as JSON
json_path = os.path.join(output_dir, 'model_architecture.json')
model_json = model.to_json()
with open(json_path, 'w') as f:
    f.write(model_json)
print(f"Model architecture saved to: {json_path}")

# Save model weights
weights_path = os.path.join(output_dir, 'model_weights.h5')
model.save_weights(weights_path)
print(f"Model weights saved to: {weights_path}")

print(f"\nAll model files saved in '{output_dir}' directory")


Model saved to: ./models\emotion_detection_model.h5
Label mapping saved to: ./models\emotion_label_encoder.pkl
Model architecture saved to: ./models\model_architecture.json
Model weights saved to: ./models\model_weights.h5

All model files saved in './models' directory


### **7. YOLO Integration for Face Detection**

#### 7.1. Load YOLO for Face Detection

In [20]:
# Use pre-trained face detector from OpenCV (simpler and faster)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

print("Face detector (Haar Cascade) loaded successfully")

# Alternative: Download YOLOv5 face detection model (uncomment if needed)
# model_yolo = torch.hub.load('ultralytics/yolov5', 'custom', 
#                             path='path/to/yolov5_face_weights.pt')
# print("YOLO face detector loaded successfully")

Face detector (Haar Cascade) loaded successfully


#### 7.2. Emotion Detection on Image

In [21]:
def detect_emotions_in_image(image_path, emotion_model, face_cascade, emotion_classes):
    """Detect emotions in an image using face detection + CNN"""
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Cannot read image from {image_path}")
        return None, []
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    results = []
    
    for (x, y, w, h) in faces:
        # Extract face region
        face_region = img_rgb[y:y+h, x:x+w]
        
        # Preprocess face for emotion model
        face_resized = cv2.resize(face_region, (IMG_SIZE, IMG_SIZE))
        face_normalized = face_resized / 255.0
        face_input = np.expand_dims(face_normalized, axis=0)
        
        # Predict emotion
        emotion_pred = emotion_model.predict(face_input, verbose=0)
        emotion_idx = np.argmax(emotion_pred[0])
        emotion = emotion_map[emotion_idx]
        confidence = emotion_pred[0][emotion_idx]
        
        results.append({
            'position': (x, y, w, h),
            'emotion': emotion,
            'confidence': confidence
        })
        
        # Draw rectangle and label on image
        cv2.rectangle(img_rgb, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f"{emotion} ({confidence:.2f})"
        cv2.putText(img_rgb, label, (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return img_rgb, results

# Test on an image from the test set
test_emotion = EMOTION_CLASSES[0]  # ✅ Select FIRST emotion from the list
test_emotion_path = os.path.join(test_dir, test_emotion)
test_image = os.listdir(test_emotion_path)[0]  # ✅ Also select FIRST image
test_image_path = os.path.join(test_emotion_path, test_image)

result_img, detections = detect_emotions_in_image(test_image_path, model, face_cascade, EMOTION_CLASSES)

if result_img is not None:
    plt.figure(figsize=(10, 8))
    plt.imshow(result_img)
    plt.title("Emotion Detection Results from Test Set", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print(f"Detected {len(detections)} face(s)")
    for i, detection in enumerate(detections):
        print(f"  Face {i+1}: {detection['emotion'].capitalize()} (confidence: {detection['confidence']:.4f})")

Detected 1 face(s)
  Face 1: Happiness (confidence: 0.2819)


### **8. Real-Time Emotion Detection**

#### 8.1. Real-Time Emotion Detection from Webcam

In [ ]:
def real_time_emotion_detection(emotion_model, face_cascade, emotion_classes):
    """Real-time emotion detection from webcam"""
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Cannot access webcam")
        return
    
    print("Starting real-time emotion detection...")
    print("Press 'q' to exit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Cannot read from webcam")
            break
        
        # Flip frame horizontally for mirror effect
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            # Extract and preprocess face
            face_region = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_region, (IMG_SIZE, IMG_SIZE))
            face_normalized = face_resized / 255.0
            face_input = np.expand_dims(face_normalized, axis=0)
            
            # Predict emotion
            emotion_pred = emotion_model.predict(face_input, verbose=0)
            emotion_idx = np.argmax(emotion_pred[0])  # ✅ Access [0] for batch
            emotion = emotion_map[emotion_idx]
            confidence = emotion_pred[0][emotion_idx]  # ✅ Access [0] for batch first
            
            # Draw rectangle
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            
            # Add label
            label = f"{emotion} ({confidence:.2f})"
            cv2.putText(frame, label, (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Display frame
        cv2.imshow('Real-Time Emotion Detection (Press q to exit)', frame)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Real-time detection stopped")

# Uncomment to run real-time detection
# real_time_emotion_detection(model, face_cascade, EMOTION_CLASSES)


Starting real-time emotion detection...
Press 'q' to exit
Real-time detection stopped


### **9. Advanced YOLO Integration**

#### 9.1. Using YOLOv5 for Better Face Detection

In [23]:
"""
def detect_emotions_yolov5(image_path, emotion_model, yolo_model, emotion_classes):
     # Detect emotions using YOLOv5 for face detection
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Cannot read image from {image_path}")
        return None, []
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # YOLOv5 detection
    results = yolo_model(img_rgb)
    detections = results.xyxy[0].cpu().numpy()  # ✅ Access [0] for first batch
    
    emotion_results = []
    
    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        if conf > 0.5:  # Confidence threshold
            # Extract face region
            face_region = img_rgb[y1:y2, x1:x2]
            
            # Preprocess for emotion model
            face_resized = cv2.resize(face_region, (IMG_SIZE, IMG_SIZE))
            face_normalized = face_resized / 255.0
            face_input = np.expand_dims(face_normalized, axis=0)
            
            # Predict emotion
            emotion_pred = emotion_model.predict(face_input, verbose=0)
            emotion_idx = np.argmax(emotion_pred[0])  # ✅ Access [0] for batch
            emotion = emotion_map[emotion_idx]
            emotion_conf = emotion_pred[0][emotion_idx]  # ✅ Access [0] for batch first
            
            emotion_results.append({
                'bbox': (x1, y1, x2, y2),
                'emotion': emotion,
                'confidence': float(emotion_conf)  # ✅ Convert to float for JSON serialization
            })
            
            # Draw on image
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{emotion} ({emotion_conf:.2f})"
            cv2.putText(img_rgb, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return img_rgb, emotion_results


# To use YOLOv5, first load the model:
# First, download YOLOv5 face weights
# You can use a pre-trained face detection model like:
# !wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt

# Then load it:
model_yolo = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt')

# Or use a face-specific YOLOv5 model:
# model_yolo = torch.hub.load('ultralytics/yolov5', 'custom', 
#                             path='path/to/yolov5_face_weights.pt')

# Then test:
result_img, detections = detect_emotions_yolov5(test_image_path, model, model_yolo, EMOTION_CLASSES)
if result_img is not None:
plt.figure(figsize=(12, 10))
plt.imshow(result_img)
plt.title("Emotion Detection with YOLOv5", fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()
print(f"Detected {len(detections)} face(s)")
"""

'\ndef detect_emotions_yolov5(image_path, emotion_model, yolo_model, emotion_classes):\n     # Detect emotions using YOLOv5 for face detection\n    \n    # Read image\n    img = cv2.imread(image_path)\n    if img is None:\n        print(f"Error: Cannot read image from {image_path}")\n        return None, []\n    \n    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n    \n    # YOLOv5 detection\n    results = yolo_model(img_rgb)\n    detections = results.xyxy[0].cpu().numpy()  # ✅ Access [0] for first batch\n    \n    emotion_results = []\n    \n    for detection in detections:\n        x1, y1, x2, y2, conf, cls = detection\n        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)\n        \n        if conf > 0.5:  # Confidence threshold\n            # Extract face region\n            face_region = img_rgb[y1:y2, x1:x2]\n            \n            # Preprocess for emotion model\n            face_resized = cv2.resize(face_region, (IMG_SIZE, IMG_SIZE))\n            face_normalized = fac

### **10. Save and Load Models**

#### 10.1. Save Models